## Import modules

In [58]:
from pyspark.sql.types import *
from pyspark.sql.functions import col
from pyspark.sql.functions import year, sum

StatementMeta(, 3f4436d7-d155-4b37-b4cf-100fb7485954, 62, Finished, Available)

## Input parameters

Provide the folder URL details for silver and gold layers of the ESG lake

Valid Inputs for the input list `reportsToGenerate`:

- BusinessMetricDataPrivacy
- BusinessMetricCommunitySupport
- BusinessMetricLaborRelations
- ResponsibleSourcingMetric
- BusinessMetricServiceDisruption
- PartyBusinessEthicsMetric
- BusinessMetricBoardOfDirectors
- Incident
- HealthSafetyTrainingMetric
- LearningEvent
- Employee


In [59]:
# Input the reports to be generated
reportsToGenerate = [
    'BusinessMetricDataPrivacy', 
    'BusinessMetricCommunitySupport',
    'BusinessMetricLaborRelations',
    'ResponsibleSourcingMetric',
    'BusinessMetricServiceDisruption',
    'PartyBusinessEthicsMetric',
    'BusinessMetricBoardOfDirectors',
    'HealthSafetyTrainingMetric',
    'LearningEvent',
    'Incident',
    'Employee']

StatementMeta(, 3f4436d7-d155-4b37-b4cf-100fb7485954, 63, Finished, Available)

## Method for importing the silver layer tables

In [60]:
# def importData(tableName):
#     df = spark.read.format("delta").load(sourceFolderUrl + '/' + tableName)
#     return df

StatementMeta(, 3f4436d7-d155-4b37-b4cf-100fb7485954, 64, Finished, Available)

## Method for exporting the gold layer tables

In [61]:
# def exportData(df, tableName):
#     print("Exporting table =", tableName)
#     df.write.format("parquet").save(destinationFolderUrl + '/' + tableName)

StatementMeta(, 3f4436d7-d155-4b37-b4cf-100fb7485954, 65, Finished, Available)

In [62]:
# df = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_PartyDataPrivacyMetric LIMIT 1000")
# display(df)

StatementMeta(, 3f4436d7-d155-4b37-b4cf-100fb7485954, 66, Finished, Available)

## Silver to Gold Transformation: `Data Privacy Metric`

In [63]:
def businessMetricDataPrivacy():
    dp = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_PartyDataPrivacyMetric")
    mt = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_DataPrivacyMetricType")
    mp = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_MetricPurpose")
    df = dp.join(mt, on="DataPrivacyMetricTypeId")
    df = df.join(mp, on="MetricPurposeId")

    selected_columns = ["DataPrivacyMetricTypeName", "PeriodStartDate", "PeriodEndDate", "NumberOfPartyDataPrivacyIncidents"]
    df = df.select(selected_columns)
    df = df.withColumnRenamed("DataPrivacyMetricTypeName", "DataPrivacyMetricType")

    dataFrames["BusinessMetricDataPrivacy"] = df

StatementMeta(, 3f4436d7-d155-4b37-b4cf-100fb7485954, 67, Finished, Available)

## Silver to Gold Transformation: `Community Support Metric`

In [64]:
def businessMetricCommunitySupport():
    pbt = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_PartyBusinessMetric")
    bm = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_BusinessMetric")
    bmt = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_BusinessMetricType")

    df = pbt.join(bm, on="BusinessMetricId")
    df = df.join(bmt, on="BusinessMetricTypeId")

    selected_columns = ["BusinessMetricName", "PartyBusinessMetricValue", "PeriodStartDate", "PeriodEndDate"]
    df = df.filter(df.BusinessMetricTypeName == "Community support").select(selected_columns)

    df = df.withColumnRenamed("BusinessMetricName", "metricType")
    df = df.withColumn("year", year(df["PeriodStartDate"]))
    df = df.groupBy("year", "metricType").agg(sum("PartyBusinessMetricValue").alias("metricValue"))

    dataFrames["BusinessMetricCommunitySupport"] =  df


StatementMeta(, 3f4436d7-d155-4b37-b4cf-100fb7485954, 68, Finished, Available)

## Silver to Gold Transformation: `Labor Relations Metric`

In [65]:
def businessMetricLaborRelations():
    pbt = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_PartyBusinessMetric")
    bm = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_BusinessMetric")
    bmt = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_BusinessMetricType")

    df = pbt.join(bm, on="BusinessMetricId")
    df = df.join(bmt, on="BusinessMetricTypeId")

    selected_columns = ["BusinessMetricName", "PartyBusinessMetricValue", "PeriodStartDate", "PeriodEndDate"]
    df = df.filter(df.BusinessMetricTypeName == "Labor relations").select(selected_columns)

    df = df.withColumnRenamed("BusinessMetricName", "metricType")
    df = df.withColumn("year", year(df["PeriodStartDate"]))
    df = df.groupBy("year", "metricType").agg(sum("PartyBusinessMetricValue").alias("metricValue"))

    dataFrames["BusinessMetricLaborRelations"] =  df


StatementMeta(, 3f4436d7-d155-4b37-b4cf-100fb7485954, 69, Finished, Available)

## Silver to Gold Transformation: `Responsible Sourcing Metric`

In [66]:
def responsibleSourcingMetric():
    prs = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_PartyResponsibleSourcing")
    rst = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_ResponsibleSourcingType")

    df = prs.join(rst, on="ResponsibleSourcingTypeId")

    selected_columns = ["ResponsibleSourcingTypeName", "NumberOfPartyResponsibleSourcingVendors", "PeriodStartDate", "PeriodEndDate"]

    df = df.withColumnRenamed("ResponsibleSourcingTypeName", "ResponsibleSourcingType")
    df = df.withColumn("Year", year(df["PeriodStartDate"]))
    df = df.groupBy("Year", "ResponsibleSourcingType").agg(sum("NumberOfPartyResponsibleSourcingVendors").alias("NumberOfPartyResponsibleSourcingVendors"))

    dataFrames["ResponsibleSourcingMetric"] =  df


StatementMeta(, 3f4436d7-d155-4b37-b4cf-100fb7485954, 70, Finished, Available)

## Silver to Gold Transformation: `Service Disruption Metric`

In [67]:
def businessMetricServiceDisruption():
    prs = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_PartyCustomerGroupServiceDisruption")
    rst = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_ServiceDisruptionType")

    df = prs.join(rst, on="ServiceDisruptionTypeId")

    selected_columns = ["ServiceDisruptionTypeName", "NumberOfPartyCustomerGroupServiceDisruptions", "PeriodStartDate", "PeriodEndDate"]

    df = df.select(selected_columns)
    df = df.withColumnRenamed("ServiceDisruptionTypeName", "ServiceDisruptionType")

    dataFrames["BusinessMetricServiceDisruption"] =  df


StatementMeta(, 3f4436d7-d155-4b37-b4cf-100fb7485954, 71, Finished, Available)

## Silver to Gold Transformation: `Business Ethics Metric`

In [68]:
def partyBusinessEthicsMetric():
    df1 = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_PartyGeographicAreaBusinessEthicsMetric")
    df2 = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_GeographicArea")
    df3 = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_BusinessEthicsMetricType")

    df = df1.join(df2, on="GeographicAreaId")
    df = df.join(df3, on="BusinessEthicsMetricTypeId")

    selected_columns = ["BusinessEthicsMetricTypeName", "PeriodStartDate", "PeriodEndDate", "PartyGeographicAreaBusinessEthicsOccurrencesNumber",
    "NumberOfPartyGeographicAreaBusinessEthicsActionsLeadingToLegalAction", "PartyGeographicAreaBusinessEthicsLegalActionsSuitsAmount", 
    "PartyGeographicAreaBusinessEthicsLegalActionSuitsPaidAmount", "PartyGeographicAreaBusinessEthicsLegalActionsCostsAmount",
    "PartyGeographicAreaBusinessEthicsFinesAmount", "GeographicAreaId", "GeographicAreaName"]
    df = df.select(selected_columns)

    renameColumns = [
        ["BusinessEthicsMetricTypeName", "BusinessEthicsMetricType"],
        ["PartyGeographicAreaBusinessEthicsOccurrencesNumber", "BusinessEthicsOccurrencesNumber"],
        ["NumberOfPartyGeographicAreaBusinessEthicsActionsLeadingToLegalAction", "BusinessEthicsActionsLeadingToLegalAction"],
        ["PartyGeographicAreaBusinessEthicsLegalActionsSuitsAmount", "BusinessEthicsLegalActionsSuitsAmount"],
        ["PartyGeographicAreaBusinessEthicsLegalActionSuitsPaidAmount", "BusinessEthicsLegalActionSuitsPaidAmount"],
        ["PartyGeographicAreaBusinessEthicsLegalActionsCostsAmount", "BusinessEthicsLegalActionsCostsAmount"],
        ["PartyGeographicAreaBusinessEthicsFinesAmount", "BusinessEthicsFinesAmount"]
    ]
    for col in renameColumns:
        df = df.withColumnRenamed(col[0], col[1])
    
    dataFrames["PartyBusinessEthicsMetric"] =  df


StatementMeta(, 3f4436d7-d155-4b37-b4cf-100fb7485954, 72, Finished, Available)

## Silver to Gold Transformation: `Metric on Board of Directors`

In [69]:
def businessMetricBoardOfDirectors():
    pbt = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_PartyBusinessMetric")
    bm = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_BusinessMetric")
    bmt =  spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_BusinessMetricType")

    df = pbt.join(bm, on="BusinessMetricId")
    df = df.join(bmt, on="BusinessMetricTypeId")

    selected_columns = ["BusinessMetricName", "PartyBusinessMetricValue", "PeriodStartDate", "PeriodEndDate"]
    df = df.filter(df.BusinessMetricTypeName == "Board of directors").select(selected_columns)

    df = df.withColumnRenamed("BusinessMetricName", "metricType")
    df = df.withColumn("year", year(df["PeriodStartDate"]))
    df = df.groupBy("year", "metricType").agg(sum("PartyBusinessMetricValue").alias("metricValue"))

    df = df.groupBy("year").pivot("metricType").sum("metricValue")

    camel_case_columns = [col(col_name).alias(''.join(word.capitalize() for word in col_name.split(' '))) for col_name in df.columns]
    df = df.select(camel_case_columns)

    dataFrames["BusinessMetricBoardOfDirectors"] =  df


StatementMeta(, 3f4436d7-d155-4b37-b4cf-100fb7485954, 73, Finished, Available)

## Silver to Gold Transformation: `Incident Metric`

In [70]:
def incident():
    incident = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_Incident")
    incidentCost = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_IncidentCost")
    incidentType = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_IncidentType")
    incidentCostType = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_IncidentCostType")
    loc=spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_Location")
    con=spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_Country")

    location = loc.select(["LocationId", "CountryId"])
    country = con.select(["Iso2LetterCountryCode", "IsoCountryName", "CountryId"])
    incidentLocation = location.join(country, on="CountryId").select(["LocationId", "IsoCountryName", "Iso2LetterCountryCode"])

    incident = incident.join(incidentLocation, on="LocationId")
    incident = incident.join(incidentType, on="IncidentTypeId")
    incident = incident.drop("IncidentTypeId", "IncidentTypeDescription", "LocationId")
    incident = incident.withColumnRenamed("IncidentTypeName", "IncidentType")

    incidentCost = incidentCost.join(incidentCostType, on="IncidentCostTypeId")
    incidentCost = incidentCost.drop("IncidentCostTypeId", "IncidentCostTypeDescription")
    incidentCost = incidentCost.withColumnRenamed("IncidentCostTypeName", "IncidentCostType")

    dataFrames["Incident"] =  incident
    dataFrames["IncidentCost"] =  incidentCost

StatementMeta(, 3f4436d7-d155-4b37-b4cf-100fb7485954, 74, Finished, Available)

## Silver to Gold Transformation: `Health Safety Training Metric`

In [71]:
def healthSafetyTrainingMetric():
    df1 = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_PartyGeographicAreaHealthSafetyTrainingMetric")
    df2 = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_GeographicArea")
    df3 = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_HealthSafetyTrainingMetricType")
    df4 = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_UnitOfMeasure")

    df = df1.join(df2, on="GeographicAreaId")
    df = df.join(df3, on="HealthSafetyTrainingMetricTypeId")
    df = df.join(df4, on="UnitOfMeasureId")

    selected_columns = ["PartyId", "HealthSafetyTrainingMetricTypeName", "PeriodStartDate", "PeriodEndDate", "GeographicAreaId", 
    "GeographicAreaName", "PartyGeographicAreaHealthSafetyTrainingQuantity", "PartyGeographicAreaHealthSafetyTrainingCostAmount", "UnitOfMeasureName"]
    df = df.select(selected_columns)

    renameColumns = [
        ["HealthSafetyTrainingMetricTypeName", "HealthSafetyTrainingMetricType"],
        ["PartyGeographicAreaHealthSafetyTrainingQuantity", "PartyHealthSafetyTrainingQuantity"],
        ["PartyGeographicAreaHealthSafetyTrainingCostAmount", "PartyHealthSafetyTrainingCostAmount"],
        ["UnitOfMeasureName", "UnitOfMeasure"] 
    ]

    for col in renameColumns:
        df = df.withColumnRenamed(col[0], col[1])

    dataFrames["HealthSafetyTrainingMetric"] =  df


StatementMeta(, 3f4436d7-d155-4b37-b4cf-100fb7485954, 75, Finished, Available)

## Silver to Gold Transformation: `Learning Event Metric`

In [72]:
def learningEvent():
    df1 = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_LearningEvent")
    df2 = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_LearningEventPartyRelationshipType")
    df3 = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_LearningEventRelatedParty")
    df4 = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_LearningEventType")
    df5 = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_LearningEventClassification")

    df = df1.join(df3, on="LearningEventId")
    df = df.join(df4, on="LearningEventTypeId")
    df = df.join(df5, on="LearningEventClassificationId")
    
    renameColumns = [
        ["PeriodStartDate", "LearningEventStartTimestamp"],
        ["PeriodEndDate", "LearningEventEndTimestamp"]
    ]

    for col in renameColumns:
        df = df.withColumnRenamed(col[0], col[1])

    df = df.select(["LearningEventId", "LearningEventTypeName", "LearningEventClassificationName",
    "LearningEventStartTimestamp", "LearningEventEndTimestamp"])

    standard = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_Standard")
    standardType = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_StandardType")
    standardTraining = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_StandardTraining")

    standardsDf = standardTraining.join(standard, on="StandardId")
    standardsDf = standardsDf.join(standardType, on="StandardTypeId").select(["LearningEventId",
        "StandardTrainingPeriodStartTimestamp", "StandardTrainingPeriodEndTimestamp", "StandardTypeName"])

    renameColumns = [
        ["StandardTrainingPeriodStartTimestamp", "StandardTrainingTypeStartTimestamp"],
        ["StandardTrainingPeriodEndTimestamp", "StandardTrainingTypeEndTimestamp"],
        ["StandardTypeName", "StandardTrainingType"]
    ]

    for col in renameColumns:
        standardsDf = standardsDf.withColumnRenamed(col[0], col[1])

    df = df.join(standardsDf, on="LearningEventId")

    dataFrames["LearningEvent"] =  df


StatementMeta(, 3f4436d7-d155-4b37-b4cf-100fb7485954, 76, Finished, Available)

## Silver to Gold Transformation: `Employee Metric`

In [73]:
def employee():
    employee = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_Employee")
    employeeRelatedParty = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_EmployeeRelatedParty")
    employeePartyRelationshipType = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_EmployeePartyRelationshipType")
    party = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_Party")

    employeeLocation =spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_EmployeeLocation")
    location = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_Location")

    racialCategory = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_RacialCategory")
    gender = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_Gender")

    country = spark.sql("SELECT * FROM #LAKEHOUSE_SILVER_NAME#.SG_Country")

    employeeRelationship = employeeRelatedParty.join(party, on="PartyId").select(["EmployeeId", "PartyName", "EmployeePartyRelationshipTypeId"])
    employeeRelationship = employeeRelationship.withColumnRenamed("PartyName", "EmployeeRelatedPartyName")

    df = employee.join(employeeRelationship, on="EmployeeId")
    df = df.join(employeePartyRelationshipType, on="EmployeePartyRelationshipTypeId")
    df = df.join(employeeLocation, on="EmployeeId")

    df = df.join(location, on="LocationId")

    df = df.join(racialCategory, on="RacialCategoryId")
    df = df.join(gender, on="GenderId")

    df = df.join(country, on="CountryId")

    selected_columns = ["EmployeeId", "EmployeeRelatedPartyName", "EmployeeName", "DateOfBirth", "EmployeeHireDate", "HighlyCompensatedEmployeeIndicator", "GenderName",
    "RacialCategoryName", "EmployeePartyRelationshipTypeName", "Iso2LetterCountryCode", "IsoCountryName"]
    df = df.select(selected_columns)

    renameColumns = [
        ["GenderName", "Gender"],
        ["RacialCategoryName", "RacialCategory"],
        ["EmployeePartyRelationshipTypeName", "EmployeePartyRelationshipType"],
    ]

    for col in renameColumns:
        df = df.withColumnRenamed(col[0], col[1])
    
    dataFrames["Employee"] =  df


StatementMeta(, 3f4436d7-d155-4b37-b4cf-100fb7485954, 77, Finished, Available)

## Aggregate, de-normalize and export tables to gold layer

In [74]:
reportsToMethodMap = [
    ('BusinessMetricDataPrivacy', businessMetricDataPrivacy),
    ('BusinessMetricCommunitySupport', businessMetricCommunitySupport),
    ('BusinessMetricLaborRelations', businessMetricLaborRelations),
    ('ResponsibleSourcingMetric', responsibleSourcingMetric),
    ('BusinessMetricServiceDisruption', businessMetricServiceDisruption),
    ('PartyBusinessEthicsMetric', partyBusinessEthicsMetric),
    ('BusinessMetricBoardOfDirectors', businessMetricBoardOfDirectors),
    ('Incident', incident),
    ('HealthSafetyTrainingMetric', healthSafetyTrainingMetric),
    ('LearningEvent', learningEvent),
    ('Employee', employee)
]

def getCalculationFunctionForReport(reportName):
    for x in reportsToMethodMap:
        if x[0] == reportName:
            return x[1]
    raise ValueError('Invalid report name found in reportsToMethodMap parameter. Invalid input = ' + reportName)


dataFrames = {}

for reportName in reportsToGenerate:
    #print("Calculating report =", reportName)
    calculationFunction = getCalculationFunctionForReport(reportName)
    calculationFunction()





StatementMeta(, 3f4436d7-d155-4b37-b4cf-100fb7485954, 78, Finished, Available)

SynapseWidget(Synapse.DataFrame, 8003518e-ee27-44c1-bc49-91b86d0d86a5)

In [85]:
TableList=["TotalIncidentCost_By_IncidentType","TotalIncident","Employee","LearningEvent","TotalHealthSafety_TrainingCost","TotalBoardExpenses_OfDirectors",
"TotalAmount_By_PartyEthics","TotalNumberOfPartyCustomerGroup_ServiceDisruptionType","TotalNumberOf_ResponsibleSourcing_ByType",
"TotalMetricValue_By_LaborRelations","TotalBusinessMetric_By_CommunitySupport","TotalNumberOfDataPrivacyIncidents_BY_PrivacyType"]

#outputs=[BusinessMetricDataPrivacy,BusinessMetricCommunitySupport,BusinessMetricLaborRelations,ResponsibleSourcingMetric,BusinessMetricServiceDisruption,PartyBusinessEthicsMetric,BusinessMetricBoardOfDirectors,HealthSafetyTrainingMetric,LearningEvent,Incident,IncidentCost,Employee]
for tableName in dataFrames:
    print(tableName)
    if tableName=='BusinessMetricDataPrivacy':
        dataFrames['BusinessMetricDataPrivacy'].write.mode("overwrite").format("delta").saveAsTable('#LAKEHOUSE_GOLD_NAME#.GS_'+'TotalNumberOfDataPrivacyIncidents_BY_PrivacyType')
    elif tableName=='BusinessMetricCommunitySupport':
        dataFrames['BusinessMetricCommunitySupport'].write.mode("overwrite").format("delta").saveAsTable('#LAKEHOUSE_GOLD_NAME#.SG_'+'TotalBusinessMetric_By_CommunitySupport')
    elif tableName=='BusinessMetricLaborRelations':
        dataFrames['BusinessMetricLaborRelations'].write.mode("overwrite").format("delta").saveAsTable('#LAKEHOUSE_GOLD_NAME#.SG_'+'TotalMetricValue_By_LaborRelations')
    elif tableName=='ResponsibleSourcingMetric':
        dataFrames['ResponsibleSourcingMetric'].write.mode("overwrite").format("delta").saveAsTable('#LAKEHOUSE_GOLD_NAME#.SG_'+'TotalNumberOf_ResponsibleSourcing_ByType')
    elif tableName=='BusinessMetricServiceDisruption':
        dataFrames['BusinessMetricServiceDisruption'].write.mode("overwrite").format("delta").saveAsTable('#LAKEHOUSE_GOLD_NAME#.SG_'+'TotalNumberOfPartyCustomerGroup_ServiceDisruptionType')
    elif tableName=='PartyBusinessEthicsMetric':
        dataFrames['PartyBusinessEthicsMetric'].write.mode("overwrite").format("delta").saveAsTable('#LAKEHOUSE_GOLD_NAME#.SG_'+'TotalAmount_By_PartyEthics')
    elif tableName=='BusinessMetricBoardOfDirectors':
        dataFrames['BusinessMetricBoardOfDirectors'].write.mode("overwrite").format("delta").saveAsTable('#LAKEHOUSE_GOLD_NAME#.SG_'+'TotalBoardExpenses_OfDirectors')
    elif tableName=='HealthSafetyTrainingMetric':
        dataFrames['HealthSafetyTrainingMetric'].write.mode("overwrite").format("delta").saveAsTable('#LAKEHOUSE_GOLD_NAME#.SG_'+'TotalHealthSafety_TrainingCost')

    elif tableName=='LearningEvent':
        dataFrames['LearningEvent'].write.mode("overwrite").format("delta").saveAsTable('#LAKEHOUSE_SILVER_NAME#.SG_'+'Learning_Event')
    
    elif tableName=='Incident':
        dataFrames['Incident'].write.mode("overwrite").format("delta").saveAsTable('#LAKEHOUSE_GOLD_NAME#.SG_'+'TotalIncident')

    
    elif tableName=='IncidentCost':
        dataFrames['IncidentCost'].write.mode("overwrite").format("delta").saveAsTable('#LAKEHOUSE_GOLD_NAME#.SG_'+'TotalIncidentCost_By_IncidentType')

        
    elif tableName=='Employee':
        dataFrames['Employee'].write.mode("overwrite").format("delta").saveAsTable('#LAKEHOUSE_SILVER_NAME#.SG_'+'Employee_Data')

print('Data loaded into lakehouse (Gold Layer)')

    #display()
    #dataFrames[tableName].write.mode("overwrite").mode("delta").saveAsTable(TableList)


#     exportData(dataFrames[tableName], tableName)

StatementMeta(, 3f4436d7-d155-4b37-b4cf-100fb7485954, 89, Finished, Available)

BusinessMetricDataPrivacy
BusinessMetricCommunitySupport
BusinessMetricLaborRelations
ResponsibleSourcingMetric
BusinessMetricServiceDisruption
PartyBusinessEthicsMetric
BusinessMetricBoardOfDirectors
HealthSafetyTrainingMetric
LearningEvent
Incident
IncidentCost
Employee
Data loaded into lakehouse (Gold Layer)
